In [18]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [19]:
States_df = pd.read_csv('../data/States_df.csv')

In [6]:
States_df = (
    States_df
    .drop(columns = 'Party')
    .rename(columns={'Rcpts': 'Raised', 'Result': 'Outcome'})
)

In [7]:
States_df.sample(3)

,District,State,cid,FirstLastP,Raised,Spent,PACs,Indivs,Cand,Other,...,VotePercent,Outcome,CRPICO,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote
904,2,Oregon,N00046681,Patrick Shane Archer (I),0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,,O,NaN,NaN,2020-05-19 00:00:00 +0000,NaN,0,2,N
312,10,Georgia,N00043150,Tabitha Johnson-Green (D),14807.96,12213.57,0.0,14997.45,0.0,-189.49,...,37.7,L,C,NaN,NaN,2020-05-19 00:00:00 +0000,,0,2,N
537,6,Michigan,N00047740,Jeff Depoy (L),0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,,C,NaN,NaN,2020-08-04 00:00:00 +0000,NaN,0,2,N


In [10]:
data = States_df[['Raised', 'Spent', 'Outcome']]